In [ ]:
# Import necessary libraries
import copy
import numpy as np
import random

In [7]:
import random
import copy
from typing import List, TypeVar

PuzzleType = TypeVar('PuzzleType')

class Puzzlegeneration:
    initial_configs: List[PuzzleType]

    def __init__(self):
        pass

    def get_puzzle(self, index: int) -> PuzzleType:
        if not hasattr(self, 'initial_configs'):
             raise AttributeError("'initial_configs' attribute is not set. Cannot get puzzle.")

        if 0 <= index < len(self.initial_configs):
            return copy.deepcopy(self.initial_configs[index])
        else:
            raise IndexError("Puzzle index out of range.")

    @staticmethod
    def generate_random_sudoku() -> List[List[int]]:
        base = 3
        side = base * base
        random.seed(42)

        def pattern(r, c):
            return (base * (r % base) + r // base + c) % side

        def shuffle(s):
            return random.sample(s, len(s))

        r_base = range(base)
        rows = [g * base + r for g in shuffle(r_base) for r in shuffle(r_base)]
        cols = [g * base + c for g in shuffle(r_base) for c in shuffle(r_base)]
        nums = shuffle(range(1, side + 1))

        board = [[nums[pattern(r, c)] for c in cols] for r in rows]

        squares = side * side
        empties = random.randint(45, 55)

        for _ in range(empties):
            row_idx = random.randint(0, side - 1)
            col_idx = random.randint(0, side - 1)
            board[row_idx][col_idx] = 0

        return board

In [2]:
def print_puzzle(sudoku):
    for i, row in enumerate(sudoku):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        for j, cell in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            print(cell, end=" ")
        print()

In [8]:
import time
import copy
import random
from typing import List, Tuple, Dict, Optional, Set, NewType


# Type definitions for clarity
Coord = Tuple[int, int]
Domain = List[int]
Domains = Dict[Coord, Domain]
Assignment = Dict[Coord, int]
Constraint = Tuple[Coord, Coord]
Puzzle = List[List[int]]

class SudokuCSP:
    """Represents Sudoku as a Constraint Satisfaction Problem."""

    def __init__(self, puzzle: Puzzle):
        self.puzzle: Puzzle = puzzle
        self.variables: List[Coord] = [
            (r, c) for r in range(9) for c in range(9) if puzzle[r][c] == 0
        ]
        self.domains: Domains = {var: list(range(1, 10)) for var in self.variables}
        # Generate and store all binary constraints
        self.constraints: List[Constraint] = self._generate_constraints()

    def _generate_constraints(self) -> List[Constraint]:
        """Generates unique binary constraints (pairs of cells in same row, col, or box)."""
        constraints_set: Set[Constraint] = set()
        for r in range(9):
            for c in range(9):
                cell1: Coord = (r, c)
                # Row constraints (check cells to the right)
                for c2 in range(c + 1, 9):
                    # Add tuple of sorted coords to ensure uniqueness ((r,c), (r,c2))
                    constraints_set.add(tuple(sorted((cell1, (r, c2)))))
                # Column constraints (check cells below)
                for r2 in range(r + 1, 9):
                    # Add tuple of sorted coords ((r,c), (r2,c))
                    constraints_set.add(tuple(sorted((cell1, (r2, c)))))
                # Box constraints (check cells after cell1 within the same 3x3 box)
                box_start_r, box_start_c = 3 * (r // 3), 3 * (c // 3)
                for r2 in range(box_start_r, box_start_r + 3):
                    for c2 in range(box_start_c, box_start_c + 3):
                        cell2: Coord = (r2, c2)
                        # Add constraint only if cell2 > cell1 lexicographically
                        # This avoids duplicate pairs and self-constraints (cell1, cell1)
                        if cell2 > cell1:
                             constraints_set.add(tuple(sorted((cell1, cell2))))

        return list(constraints_set) # Return as list per original structure

    def is_valid(self, variable: Coord, value: int, assignment: Assignment) -> bool:
        """
        Checks if assigning 'value' to 'variable' is consistent with the current
        'assignment' and the fixed numbers in the original puzzle.
        """
        r, c = variable

        # 1. Check against other *assigned* variables using pre-computed constraints
        for (v1, v2) in self.constraints:
            if v1 == variable and v2 in assignment and assignment[v2] == value:
                return False # Conflict with assigned neighbor
            if v2 == variable and v1 in assignment and assignment[v1] == value:
                return False # Conflict with assigned neighbor

        # 2. Check against fixed numbers in the original puzzle (essential for correctness)
        # Check row
        for col_check in range(9):
             if col_check != c and self.puzzle[r][col_check] == value:
                 return False
        # Check column
        for row_check in range(9):
             if row_check != r and self.puzzle[row_check][c] == value:
                 return False
        # Check 3x3 box
        box_start_r, box_start_c = 3 * (r // 3), 3 * (c // 3)
        for row_check in range(box_start_r, box_start_r + 3):
            for col_check in range(box_start_c, box_start_c + 3):
                 if (row_check, col_check) != variable and self.puzzle[row_check][col_check] == value:
                     return False

        # If no conflicts found
        return True

    def backtracking_search(self) -> Optional[Assignment]:
        """Initializes and starts the backtracking search."""
        # Start assignment with the fixed numbers from the puzzle
        initial_assignment: Assignment = {
            (r, c): self.puzzle[r][c]
            for r in range(9) for c in range(9) if self.puzzle[r][c] != 0
        }
        return self.simple_backtrack(initial_assignment)

    def simple_backtrack(self, assignment: Assignment) -> Optional[Assignment]:
        """Recursive backtracking function."""
        # Base case: All cells are assigned (81 total)
        if len(assignment) == 81:
            return assignment # Solution found

        # Select the next unassigned variable (simple ordered selection)
        var: Optional[Coord] = None
        for v in self.variables: # self.variables only contains initially empty cells
            if v not in assignment:
                var = v
                break # Pick the first one found

        # Should find an unassigned variable if assignment isn't complete
        if var is None:
             # This case should ideally not be reached if len(assignment) < 81
             return None # Indicates failure or inconsistent state

        # Try values from the domain [1..9] for the selected variable
        for value in self.domains[var]: # self.domains[var] is always list(range(1, 10)) here
            if self.is_valid(var, value, assignment):
                assignment[var] = value # Place the value
                result = self.simple_backtrack(assignment) # Recurse
                if result is not None:
                    return result # Pass solution up the call stack
                # If recursion returned None, backtrack: remove assignment
                del assignment[var]

        # If no value from the domain worked for 'var', return failure
        return None

    def print_sudoku(self, assignment: Assignment):
        """Prints the solved Sudoku board derived from the assignment."""
        # Create a full board representation from the assignment
        solved_board: Puzzle = [[0 for _ in range(9)] for _ in range(9)]
        for (r, c), value in assignment.items():
            solved_board[r][c] = value

        # Use the common print function
        print_puzzle(solved_board)


# --- Testing Backtracking (Preserving original testing logic) ---
initial_config = Puzzlegeneration()
times: List[float] = []

print("--- Starting Basic Backtracking Test ---")

num_runs = 10
for i in range(num_runs):
    run_num = i + 1
    # Generate puzzle (same puzzle each time due to InitialConfig's fixed seed)
    puzzle: Puzzle = initial_config.generate_random_sudoku()

    print(f"Problem Puzzle (Run {run_num}/{num_runs}):")
    # Print the initial puzzle state using the standalone function
    print_puzzle(puzzle)

    # Create solver instance for the puzzle
    solver = SudokuCSP(puzzle)

    # Time the solution process
    start_time: float = time.time()
    solution: Optional[Assignment] = solver.backtracking_search()
    end_time: float = time.time()
    elapsed_time: float = end_time - start_time
    times.append(elapsed_time)

    # Print results for the current run
    print(f"Run {run_num} Result:")
    if solution:
        print(f"Solution Found ({elapsed_time:.6f} seconds):")
        solver.print_sudoku(solution) # Print the solved board
    else:
        # This should generally not happen for puzzles generated by InitialConfig
        print(f"No solution found ({elapsed_time:.6f} seconds)")

# Calculate and print average time across all runs
if times:
    avg_time: float = sum(times) / len(times)
    print(f"Average time for Basic Backtracking ({num_runs} runs on the same puzzle): {avg_time:.8f} seconds")
else:
    print("No runs were performed.")

print("--- End of Basic Backtracking Test ---")

--- Starting Basic Backtracking Test ---
Problem Puzzle (Run 1/10):
2 3 1 | 7 5 9 | 6 . 8 
. 7 . | . . . | . 1 . 
6 8 4 | . . 1 | . . 7 
- - - - - - - - - - - -
. 6 . | 2 . . | 4 . 5 
9 . 7 | 5 4 . | . . 6 
. 5 . | . . . | . 7 2 
- - - - - - - - - - - -
7 . . | 9 . . | 3 . 4 
3 4 6 | 1 7 2 | 8 5 9 
. . 5 | . . 6 | 7 . . 

Run 1 Result:
Solution Found (0.263254 seconds):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - - - - - - -
7 1 2 | 9 8 5 | 3 6 4 
3 4 6 | 1 7 2 | 8 5 9 
8 9 5 | 4 3 6 | 7 2 1 

Problem Puzzle (Run 2/10):
2 3 1 | 7 5 9 | 6 . 8 
. 7 . | . . . | . 1 . 
6 8 4 | . . 1 | . . 7 
- - - - - - - - - - - -
. 6 . | 2 . . | 4 . 5 
9 . 7 | 5 4 . | . . 6 
. 5 . | . . . | . 7 2 
- - - - - - - - - - - -
7 . . | 9 . . | 3 . 4 
3 4 6 | 1 7 2 | 8 5 9 
. . 5 | . . 6 | 7 . . 

Run 2 Result:
Solution Found (0.138814 seconds):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 

## Forward Checking with Back Tracking

In [10]:
import time
import copy
from typing import List, Tuple, Dict, Optional, Set, NewType

# Type definitions
Coord = Tuple[int, int]
Domain = List[int]
Domains = Dict[Coord, Domain]
Assignment = Dict[Coord, int]
Constraint = Tuple[Coord, Coord]
Puzzle = List[List[int]]
Neighbors = Dict[Coord, List[Coord]]

class SudokuCSP:
    # Sudoku CSP solver using Backtracking with Forward Checking

    def __init__(self, puzzle: Puzzle):
        self.puzzle: Puzzle = puzzle
        self.variables: List[Coord] = [
            (r, c) for r in range(9) for c in range(9) if puzzle[r][c] == 0
        ]
        self.domains: Domains = {var: list(range(1, 10)) for var in self.variables}
        all_constraints: List[Constraint] = self._generate_constraints()
        self.constraints: List[Constraint] = all_constraints
        self.neighbors: Neighbors = {var: [] for var in self.variables}
        variable_set = set(self.variables)
        for (v1, v2) in all_constraints:
            if v1 in variable_set and v2 in variable_set:
                self.neighbors[v1].append(v2)
                self.neighbors[v2].append(v1)

    def _generate_constraints(self) -> List[Constraint]:
        constraints_set: Set[Constraint] = set()
        for r in range(9):
            for c in range(9):
                cell1: Coord = (r, c)
                for c2 in range(c + 1, 9):
                    constraints_set.add(tuple(sorted((cell1, (r, c2)))))
                for r2 in range(r + 1, 9):
                    constraints_set.add(tuple(sorted((cell1, (r2, c)))))
                box_start_r, box_start_c = 3 * (r // 3), 3 * (c // 3)
                for r2 in range(box_start_r, box_start_r + 3):
                    for c2 in range(box_start_c, box_start_c + 3):
                        cell2: Coord = (r2, c2)
                        if cell2 > cell1:
                             constraints_set.add(tuple(sorted((cell1, cell2))))
        return list(constraints_set)

    def is_consistent(self, variable: Coord, value: int, assignment: Assignment) -> bool:
        r, c = variable
        for (v1, v2) in self.constraints:
            if v1 == variable and v2 in assignment and assignment[v2] == value: return False
            if v2 == variable and v1 in assignment and assignment[v1] == value: return False
        for k in range(9):
            if k != c and self.puzzle[r][k] == value: return False
        for k in range(9):
            if k != r and self.puzzle[k][c] == value: return False
        box_r, box_c = 3 * (r // 3), 3 * (c // 3)
        for br in range(box_r, box_r + 3):
            for bc in range(box_c, box_c + 3):
                if (br, bc) != variable and self.puzzle[br][bc] == value: return False
        return True

    def forward_checking(self) -> Optional[Assignment]:
        initial_assignment: Assignment = {
            (r, c): self.puzzle[r][c]
            for r in range(9) for c in range(9) if self.puzzle[r][c] != 0
        }
        current_domains: Domains = copy.deepcopy(self.domains)
        if not self._initial_domain_prune(current_domains):
            return None
        return self._fc_backtrack(initial_assignment, current_domains)

    def _initial_domain_prune(self, domains: Domains) -> bool:
        for var in list(domains.keys()):
            r, c = var
            values_to_remove = set()
            for k in range(9): values_to_remove.add(self.puzzle[r][k])
            for k in range(9): values_to_remove.add(self.puzzle[k][c])
            box_r, box_c = 3*(r//3), 3*(c//3)
            for br in range(box_r, box_r+3):
                for bc in range(box_c, box_c+3):
                    values_to_remove.add(self.puzzle[br][bc])
            values_to_remove.discard(0)
            original_domain = set(domains[var])
            new_domain = list(original_domain - values_to_remove)
            if not new_domain: return False
            domains[var] = new_domain
        return True

    def _fc_backtrack(self, assignment: Assignment, domains: Domains) -> Optional[Assignment]:
        if len(assignment) == 81: return assignment
        var: Optional[Coord] = None
        for v in self.variables:
            if v not in assignment:
                var = v
                break
        if var is None: return None
        for value in list(domains[var]):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                saved_domains: Domains = copy.deepcopy(domains)
                if self._forward_check(var, value, assignment, domains):
                    result = self._fc_backtrack(assignment, domains)
                    if result is not None: return result
                domains = saved_domains
                del assignment[var]
        return None

    def _forward_check(self, var: Coord, value: int, assignment: Assignment, domains: Domains) -> bool:
        for neighbor in self.neighbors[var]:
            if neighbor not in assignment:
                if value in domains[neighbor]:
                    domains[neighbor].remove(value)
                    if not domains[neighbor]: return False
        return True

    def print_sudoku(self, assignment: Assignment):
        solved_board: Puzzle = [[0 for _ in range(9)] for _ in range(9)]
        for (r, c), value in assignment.items():
            solved_board[r][c] = value
        print_puzzle(solved_board)




times: List[float] = []

print("--- Starting Forward Checking Test ---")
# Use current date and time
now = time.strftime("%Y-%m-%d %H:%M:%S")
print(f"Test performed on: {now}") # Print current date and time

initial_config = Puzzlegeneration()
for i in range(10):
    run_num = i + 1
    puzzle = initial_config.generate_random_sudoku()
    print(f"\nProblem Puzzle (Run {run_num}/{num_runs}):")
    print_puzzle(puzzle)

    solver = SudokuCSP(puzzle)

    start_time: float = time.time()
    solution: Optional[Assignment] = solver.forward_checking() # Run Forward Checking
    end_time: float = time.time()
    elapsed_time: float = end_time - start_time
    times.append(elapsed_time)

    print(f"Run {run_num} Result:")
    if solution:
        print(f"Solution Found ({elapsed_time:.6f} seconds):")
        solver.print_sudoku(solution)
    else:
        print(f"No solution found ({elapsed_time:.6f} seconds)")

# Calculate and print average time
if times:
    avg_time: float = sum(times) / len(times)
    print(f"\nAverage time for Forward Checking ({num_runs} runs on the hardcoded puzzle): {avg_time:.8f} seconds")
else:
    print("\nNo runs were performed.")

print("--- End of Forward Checking Test ---")



--- Starting Forward Checking Test ---
Test performed on: 2025-04-16 20:42:59

Problem Puzzle (Run 1/10):
2 3 1 | 7 5 9 | 6 . 8 
. 7 . | . . . | . 1 . 
6 8 4 | . . 1 | . . 7 
- - - - - - - - - - - -
. 6 . | 2 . . | 4 . 5 
9 . 7 | 5 4 . | . . 6 
. 5 . | . . . | . 7 2 
- - - - - - - - - - - -
7 . . | 9 . . | 3 . 4 
3 4 6 | 1 7 2 | 8 5 9 
. . 5 | . . 6 | 7 . . 

Run 1 Result:
Solution Found (0.021195 seconds):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 8 | 2 9 7 | 4 3 5 
9 2 7 | 5 4 3 | 1 8 6 
4 5 3 | 6 1 8 | 9 7 2 
- - - - - - - - - - - -
7 1 2 | 9 8 5 | 3 6 4 
3 4 6 | 1 7 2 | 8 5 9 
8 9 5 | 4 3 6 | 7 2 1 


Problem Puzzle (Run 2/10):
2 3 1 | 7 5 9 | 6 . 8 
. 7 . | . . . | . 1 . 
6 8 4 | . . 1 | . . 7 
- - - - - - - - - - - -
. 6 . | 2 . . | 4 . 5 
9 . 7 | 5 4 . | . . 6 
. 5 . | . . . | . 7 2 
- - - - - - - - - - - -
7 . . | 9 . . | 3 . 4 
3 4 6 | 1 7 2 | 8 5 9 
. . 5 | . . 6 | 7 . . 

Run 2 Result:
Solution Found (0.018655 seconds):


## Arc Consistency with Backtrack

In [ ]:
# Arc Consistency Implementation
class SudokuCSP:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.variables = [(i, j) for i in range(9) for j in range(9) if puzzle[i][j] == 0]
        self.domains = {(i, j): list(range(1, 10)) for i, j in self.variables}
        self.constraints = self.generate_constraints()
        self.neighbors = {var: [] for var in self.variables}
        for (v1, v2) in self.constraints:
            if v1 in self.variables and v2 in self.variables:
                self.neighbors[v1].append(v2)
                self.neighbors[v2].append(v1)

    def generate_constraints(self):
        constraints = []
        for i in range(9):
            for j in range(9):
                for jj in range(j + 1, 9):
                    if (i, j) != (i, jj):
                        constraints.append(((i, j), (i, jj)))
                for ii in range(i + 1, 9):
                    if (i, j) != (ii, j):
                        constraints.append(((i, j), (ii, j)))
                box_row, box_col = 3 * (i // 3), 3 * (j // 3)
                for bi in range(box_row, box_row + 3):
                    for bj in range(box_col, box_col + 3):
                        if (i, j) != (bi, bj) and (i, j) < (bi, bj):
                            constraints.append(((i, j), (bi, bj)))
        return constraints

    def is_consistent(self, variable, value, assignment):
        i, j = variable
        for (v1, v2) in self.constraints:
            if v1 == variable and v2 in assignment:
                if assignment[v2] == value:
                    return False
            if v2 == variable and v1 in assignment:
                if assignment[v1] == value:
                    return False
        return True

    def ac3(self):
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        domains = copy.deepcopy(self.domains)
        if self.run_ac3(domains):
            return self.backtrack(assignment, domains)
        return None

    def run_ac3(self, domains):
        queue = [(v1, v2) for v1, v2 in self.constraints if v1 in domains and v2 in domains]
        while queue:
            (xi, xj) = queue.pop(0)
            if self.revise(xi, xj, domains):
                if not domains[xi]:
                    return False
                for xk in self.neighbors[xi]:
                    if xk != xj and xk in domains:
                        queue.append((xk, xi))
        return True

    def revise(self, xi, xj, domains):
        revised = False
        for x in domains[xi][:]:
            if not any(self.is_consistent_pair(x, y) for y in domains[xj]):
                domains[xi].remove(x)
                revised = True
        return revised

    def is_consistent_pair(self, val1, val2):
        return val1 != val2

    def backtrack(self, assignment, domains):
        if len(assignment) == 81:
            return assignment
        unassigned = [(i, j) for i, j in self.variables if (i, j) not in assignment]
        if not unassigned:
            return None
        var = unassigned[0]
        for value in domains[var][:]:
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                result = self.backtrack(assignment, domains)
                if result is not None:
                    return result
                del assignment[var]
        return None

    def print_sudoku(self, assignment):
        board = [[self.puzzle[i][j] if self.puzzle[i][j] != 0 else 0 for j in range(9)] for i in range(9)]
        for (i, j), value in assignment.items():
            board[i][j] = value
        for i in range(9):
            if i % 3 == 0 and i != 0:
                print("- - - - - - - - - - - -")
            for j in range(9):
                if j % 3 == 0 and j != 0:
                    print("|", end=" ")
                print(board[i][j] if board[i][j] != 0 else ".", end=" ")
            print()
        print()

# Testing Arc Consistency
configs = Puzzlegeneration()
times = []
for i in range(10):
    puzzle = configs.generate_random_sudoku()
    print("Problem Puzzle:")
    print_puzzle(copy.deepcopy(puzzle))
    solver = SudokuCSP(puzzle)
    start_time = time.time()
    solution = solver.ac3()
    end_time = time.time()
    times.append(end_time - start_time)
    print(f"Run {i + 1} (Puzzle {i + 1}):")
    if solution:
        solver.print_sudoku(solution)
    else:
        print("No solution found")
avg_time = sum(times) / len(times)
print(f"Average time for Arc Consistency (AC-3) (10 runs): {avg_time:.8f} seconds")

Problem Puzzle:
2 3 1 | 7 5 9 | 6 0 8 
0 7 0 | 0 0 0 | 0 1 0 
6 8 4 | 0 0 1 | 0 0 7 
---------------------
0 6 0 | 2 0 0 | 4 0 5 
9 0 7 | 5 4 0 | 0 0 6 
0 5 0 | 0 0 0 | 0 7 2 
---------------------
7 0 0 | 9 0 0 | 3 0 4 
3 4 6 | 1 7 2 | 8 5 9 
0 0 5 | 0 0 6 | 7 0 0 
Run 1 (Puzzle 1):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - - - - - - -
7 1 2 | 9 8 5 | 3 6 4 
3 4 6 | 1 7 2 | 8 5 9 
8 9 5 | 4 3 6 | 7 2 1 

Problem Puzzle:
2 3 1 | 7 5 9 | 6 0 8 
0 7 0 | 0 0 0 | 0 1 0 
6 8 4 | 0 0 1 | 0 0 7 
---------------------
0 6 0 | 2 0 0 | 4 0 5 
9 0 7 | 5 4 0 | 0 0 6 
0 5 0 | 0 0 0 | 0 7 2 
---------------------
7 0 0 | 9 0 0 | 3 0 4 
3 4 6 | 1 7 2 | 8 5 9 
0 0 5 | 0 0 6 | 7 0 0 
Run 2 (Puzzle 2):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - 

## Minimum Remaining Value Heuristics

In [ ]:
# Minimum Remaining Value Heuristic Implementation
class SudokuCSP:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.variables = [(i, j) for i in range(9) for j in range(9) if puzzle[i][j] == 0]
        self.domains = self.initialize_domains()
        self.constraints = self.generate_constraints()
        self.neighbors = {var: [] for var in self.variables}
        for (v1, v2) in self.constraints:
            if v1 in self.variables and v2 in self.variables:
                self.neighbors[v1].append(v2)
                self.neighbors[v2].append(v1)

    def initialize_domains(self):
        domains = {(i, j): list(range(1, 10)) for i, j in self.variables}
        # Initialize domains based on constraints from pre-assigned values
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        for (i, j) in self.variables:
            row, col = i, j
            box_row, box_col = 3 * (i // 3), 3 * (j // 3)
            # Check row
            for jj in range(9):
                if (i, jj) in assignment:
                    if assignment[(i, jj)] in domains[(i, j)]:
                        domains[(i, j)].remove(assignment[(i, jj)])
            # Check column
            for ii in range(9):
                if (ii, j) in assignment:
                    if assignment[(ii, j)] in domains[(i, j)]:
                        domains[(i, j)].remove(assignment[(ii, j)])
            # Check 3x3 box
            for bi in range(box_row, box_row + 3):
                for bj in range(box_col, box_col + 3):
                    if (bi, bj) in assignment:
                        if assignment[(bi, bj)] in domains[(i, j)]:
                            domains[(i, j)].remove(assignment[(bi, bj)])
        return domains

    def generate_constraints(self):
        constraints = []
        for i in range(9):
            for j in range(9):
                for jj in range(j + 1, 9):
                    if (i, j) != (i, jj):
                        constraints.append(((i, j), (i, jj)))
                for ii in range(i + 1, 9):
                    if (i, j) != (ii, j):
                        constraints.append(((i, j), (ii, j)))
                box_row, box_col = 3 * (i // 3), 3 * (j // 3)
                for bi in range(box_row, box_row + 3):
                    for bj in range(box_col, box_col + 3):
                        if (i, j) != (bi, bj) and (i, j) < (bi, bj):
                            constraints.append(((i, j), (bi, bj)))
        return constraints

    def is_consistent(self, variable, value, assignment):
        i, j = variable
        for (v1, v2) in self.constraints:
            if v1 == variable and v2 in assignment:
                if assignment[v2] == value:
                    return False
            if v2 == variable and v1 in assignment:
                if assignment[v1] == value:
                    return False
        return True

    def mrv_heuristic(self):
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        domains = copy.deepcopy(self.domains)
        return self.backtrack_mrv(assignment, domains)

    def backtrack_mrv(self, assignment, domains):
        if len(assignment) == 81:
            return assignment
        var = self.select_mrv_variable(assignment, domains)
        if var is None:
            return None
        for value in domains[var][:]:
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                result = self.backtrack_mrv(assignment, domains)
                if result is not None:
                    return result
                del assignment[var]
        return None

    def select_mrv_variable(self, assignment, domains):
        unassigned = [(i, j) for i, j in self.variables if (i, j) not in assignment]
        if not unassigned:
            return None
        return min(unassigned, key=lambda var: len(domains[var]))

    def print_sudoku(self, assignment):
        board = [[self.puzzle[i][j] if self.puzzle[i][j] != 0 else 0 for j in range(9)] for i in range(9)]
        for (i, j), value in assignment.items():
            board[i][j] = value
        for i in range(9):
            if i % 3 == 0 and i != 0:
                print("- - - - - - - - - - - -")
            for j in range(9):
                if j % 3 == 0 and j != 0:
                    print("|", end=" ")
                print(board[i][j] if board[i][j] != 0 else ".", end=" ")
            print()
        print()

# Testing Minimum Remaining Value Heuristic
configs = Puzzlegeneration()
times = []
for i in range(10):
    puzzle = configs.generate_random_sudoku()
    print("Problem Puzzle:")
    print_puzzle(copy.deepcopy(puzzle))
    solver = SudokuCSP(puzzle)
    start_time = time.time()
    solution = solver.mrv_heuristic()
    end_time = time.time()
    times.append(end_time - start_time)
    print(f"Run {i + 1} (Puzzle {i + 1}):")
    if solution:
        solver.print_sudoku(solution)
    else:
        print("No solution found")
avg_time = sum(times) / len(times)
print(f"Average time for MRV Heuristic (10 runs): {avg_time:.8f} seconds")

Problem Puzzle:
2 3 1 | 7 5 9 | 6 0 8 
0 7 0 | 0 0 0 | 0 1 0 
6 8 4 | 0 0 1 | 0 0 7 
---------------------
0 6 0 | 2 0 0 | 4 0 5 
9 0 7 | 5 4 0 | 0 0 6 
0 5 0 | 0 0 0 | 0 7 2 
---------------------
7 0 0 | 9 0 0 | 3 0 4 
3 4 6 | 1 7 2 | 8 5 9 
0 0 5 | 0 0 6 | 7 0 0 
Run 1 (Puzzle 1):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - - - - - - -
7 1 2 | 9 8 5 | 3 6 4 
3 4 6 | 1 7 2 | 8 5 9 
8 9 5 | 4 3 6 | 7 2 1 

Problem Puzzle:
2 3 1 | 7 5 9 | 6 0 8 
0 7 0 | 0 0 0 | 0 1 0 
6 8 4 | 0 0 1 | 0 0 7 
---------------------
0 6 0 | 2 0 0 | 4 0 5 
9 0 7 | 5 4 0 | 0 0 6 
0 5 0 | 0 0 0 | 0 7 2 
---------------------
7 0 0 | 9 0 0 | 3 0 4 
3 4 6 | 1 7 2 | 8 5 9 
0 0 5 | 0 0 6 | 7 0 0 
Run 2 (Puzzle 2):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - 

## Degree Heuristics with MRV

In [ ]:
# Degree Heuristics with MRV Implementation
class SudokuCSP:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.variables = [(i, j) for i in range(9) for j in range(9) if puzzle[i][j] == 0]
        self.domains = self.initialize_domains()
        self.constraints = self.generate_constraints()
        self.neighbors = {var: [] for var in self.variables}
        for (v1, v2) in self.constraints:
            if v1 in self.variables and v2 in self.variables:
                self.neighbors[v1].append(v2)
                self.neighbors[v2].append(v1)

    def initialize_domains(self):
        domains = {(i, j): list(range(1, 10)) for i, j in self.variables}
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        for (i, j) in self.variables:
            row, col = i, j
            box_row, box_col = 3 * (i // 3), 3 * (j // 3)
            # Check row
            for jj in range(9):
                if (i, jj) in assignment and assignment[(i, jj)] in domains[(i, j)]:
                    domains[(i, j)].remove(assignment[(i, jj)])
            # Check column
            for ii in range(9):
                if (ii, j) in assignment and assignment[(ii, j)] in domains[(i, j)]:
                    domains[(i, j)].remove(assignment[(ii, j)])
            # Check 3x3 box
            for bi in range(box_row, box_row + 3):
                for bj in range(box_col, box_col + 3):
                    if (bi, bj) in assignment and assignment[(bi, bj)] in domains[(i, j)]:
                        domains[(i, j)].remove(assignment[(bi, bj)])
        return domains

    def generate_constraints(self):
        constraints = []
        for i in range(9):
            for j in range(9):
                for jj in range(j + 1, 9):
                    constraints.append(((i, j), (i, jj)))
                for ii in range(i + 1, 9):
                    constraints.append(((i, j), (ii, j)))
                box_row, box_col = 3 * (i // 3), 3 * (j // 3)
                for bi in range(box_row, box_row + 3):
                    for bj in range(box_col, box_col + 3):
                        if (i, j) != (bi, bj) and (i, j) < (bi, bj):
                            constraints.append(((i, j), (bi, bj)))
        return constraints

    def is_valid_puzzle(self):
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        for (v1, v2) in self.constraints:
            if v1 in assignment and v2 in assignment:
                if assignment[v1] == assignment[v2]:
                    return False
        return True

    def is_consistent(self, variable, value, assignment):
        for (v1, v2) in self.constraints:
            if v1 == variable and v2 in assignment:
                if assignment[v2] == value:
                    return False
            if v2 == variable and v1 in assignment:
                if assignment[v1] == value:
                    return False
        return True

    def mrv_degree_heuristic(self):
        if not self.is_valid_puzzle():
            return None
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        domains = copy.deepcopy(self.domains)
        return self.backtrack_mrv_degree(assignment, domains)

    def backtrack_mrv_degree(self, assignment, domains):
        if len(assignment) == 81:
            return assignment
        var = self.select_mrv_degree_variable(assignment, domains)
        if var is None:
            return None
        for value in domains[var][:]:
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                result = self.backtrack_mrv_degree(assignment, domains)
                if result is not None:
                    return result
                del assignment[var]
        return None

    def select_mrv_degree_variable(self, assignment, domains):
        unassigned = [(i, j) for i, j in self.variables if (i, j) not in assignment]
        if not unassigned:
            return None
        min_domain_size = min(len(domains[var]) for var in unassigned)
        candidates = [var for var in unassigned if len(domains[var]) == min_domain_size]
        return max(candidates, key=lambda var: sum(1 for n in self.neighbors[var] if n not in assignment))

    def print_sudoku(self, assignment):
        board = [[self.puzzle[i][j] if self.puzzle[i][j] != 0 else 0 for j in range(9)] for i in range(9)]
        if assignment:
            for (i, j), value in assignment.items():
                board[i][j] = value
        for i in range(9):
            if i % 3 == 0 and i != 0:
                print("- - - - - - - - - - - -")
            for j in range(9):
                if j % 3 == 0 and j != 0:
                    print("|", end=" ")
                print(board[i][j] if board[i][j] != 0 else ".", end=" ")
            print()
        print()

# Testing Degree Heuristics with MRV
configs = Puzzlegeneration()
times = []
for i in range(10):
    puzzle = configs.generate_random_sudoku()
    print("Problem Puzzle:")
    print_puzzle(copy.deepcopy(puzzle))
    solver = SudokuCSP(puzzle)
    start_time = time.time()
    solution = solver.mrv_degree_heuristic()
    end_time = time.time()
    times.append(end_time - start_time)
    print(f"Run {i + 1} (Puzzle {i + 1}):")
    if solution:
        solver.print_sudoku(solution)
    else:
        print("No solution found - Puzzle may be invalid")
avg_time = sum(times) / len(times)
print(f"Average time for MRV + Degree Heuristic (10 runs): {avg_time:.8f} seconds")

Problem Puzzle:
2 3 1 | 7 5 9 | 6 0 8 
0 7 0 | 0 0 0 | 0 1 0 
6 8 4 | 0 0 1 | 0 0 7 
---------------------
0 6 0 | 2 0 0 | 4 0 5 
9 0 7 | 5 4 0 | 0 0 6 
0 5 0 | 0 0 0 | 0 7 2 
---------------------
7 0 0 | 9 0 0 | 3 0 4 
3 4 6 | 1 7 2 | 8 5 9 
0 0 5 | 0 0 6 | 7 0 0 
Run 1 (Puzzle 1):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - - - - - - -
7 1 2 | 9 8 5 | 3 6 4 
3 4 6 | 1 7 2 | 8 5 9 
8 9 5 | 4 3 6 | 7 2 1 

Problem Puzzle:
2 3 1 | 7 5 9 | 6 0 8 
0 7 0 | 0 0 0 | 0 1 0 
6 8 4 | 0 0 1 | 0 0 7 
---------------------
0 6 0 | 2 0 0 | 4 0 5 
9 0 7 | 5 4 0 | 0 0 6 
0 5 0 | 0 0 0 | 0 7 2 
---------------------
7 0 0 | 9 0 0 | 3 0 4 
3 4 6 | 1 7 2 | 8 5 9 
0 0 5 | 0 0 6 | 7 0 0 
Run 2 (Puzzle 2):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - 

## Least Constraining Value Heuristic

In [ ]:
# Least Constraining Value Heuristic Implementation
class SudokuCSP:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.variables = [(i, j) for i in range(9) for j in range(9) if puzzle[i][j] == 0]
        self.domains = self.initialize_domains()
        self.constraints = self.generate_constraints()
        self.neighbors = {var: [] for var in self.variables}
        for (v1, v2) in self.constraints:
            if v1 in self.variables and v2 in self.variables:
                self.neighbors[v1].append(v2)
                self.neighbors[v2].append(v1)

    def initialize_domains(self):
        domains = {(i, j): list(range(1, 10)) for i, j in self.variables}
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        for (i, j) in self.variables:
            row, col = i, j
            box_row, box_col = 3 * (i // 3), 3 * (j // 3)
            # Check row
            for jj in range(9):
                if (i, jj) in assignment and assignment[(i, jj)] in domains[(i, j)]:
                    domains[(i, j)].remove(assignment[(i, jj)])
            # Check column
            for ii in range(9):
                if (ii, j) in assignment and assignment[(ii, j)] in domains[(i, j)]:
                    domains[(i, j)].remove(assignment[(ii, j)])
            # Check 3x3 box
            for bi in range(box_row, box_row + 3):
                for bj in range(box_col, box_col + 3):
                    if (bi, bj) in assignment and assignment[(bi, bj)] in domains[(i, j)]:
                        domains[(i, j)].remove(assignment[(bi, bj)])
        return domains

    def generate_constraints(self):
        constraints = []
        for i in range(9):
            for j in range(9):
                for jj in range(j + 1, 9):
                    constraints.append(((i, j), (i, jj)))
                for ii in range(i + 1, 9):
                    constraints.append(((i, j), (ii, j)))
                box_row, box_col = 3 * (i // 3), 3 * (j // 3)
                for bi in range(box_row, box_row + 3):
                    for bj in range(box_col, box_col + 3):
                        if (i, j) != (bi, bj) and (i, j) < (bi, bj):
                            constraints.append(((i, j), (bi, bj)))
        return constraints

    def is_valid_puzzle(self):
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        for (v1, v2) in self.constraints:
            if v1 in assignment and v2 in assignment:
                if assignment[v1] == assignment[v2]:
                    return False
        return True

    def is_consistent(self, variable, value, assignment):
        for (v1, v2) in self.constraints:
            if v1 == variable and v2 in assignment:
                if assignment[v2] == value:
                    return False
            if v2 == variable and v1 in assignment:
                if assignment[v1] == value:
                    return False
        return True

    def lcv_heuristic(self):
        if not self.is_valid_puzzle():
            return None
        assignment = {(i, j): self.puzzle[i][j] for i in range(9) for j in range(9) if self.puzzle[i][j] != 0}
        domains = copy.deepcopy(self.domains)
        return self.backtrack_lcv(assignment, domains)

    def backtrack_lcv(self, assignment, domains):
        if len(assignment) == 81:
            return assignment
        var = self.select_mrv_degree_variable(assignment, domains)
        if var is None:
            return None
        values = sorted(domains[var], key=lambda val: self.count_constraints(var, val, assignment, domains))
        for value in values:
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                result = self.backtrack_lcv(assignment, domains)
                if result is not None:
                    return result
                del assignment[var]
        return None

    def select_mrv_degree_variable(self, assignment, domains):
        unassigned = [(i, j) for i, j in self.variables if (i, j) not in assignment]
        if not unassigned:
            return None
        min_domain_size = min(len(domains[var]) for var in unassigned)
        candidates = [var for var in unassigned if len(domains[var]) == min_domain_size]
        return max(candidates, key=lambda var: sum(1 for n in self.neighbors[var] if n not in assignment))

    def count_constraints(self, var, value, assignment, domains):
        count = 0
        for neighbor in self.neighbors[var]:
            if neighbor not in assignment and value in domains[neighbor]:
                count += 1
        return count

    def print_sudoku(self, assignment):
        board = [[self.puzzle[i][j] if self.puzzle[i][j] != 0 else 0 for j in range(9)] for i in range(9)]
        if assignment:
            for (i, j), value in assignment.items():
                board[i][j] = value
        for i in range(9):
            if i % 3 == 0 and i != 0:
                print("- - - - - - - - - - - -")
            for j in range(9):
                if j % 3 == 0 and j != 0:
                    print("|", end=" ")
                print(board[i][j] if board[i][j] != 0 else ".", end=" ")
            print()
        print()

# Testing Least Constraining Value Heuristic
configs = Puzzlegeneration()
times = []
for i in range(10):
    puzzle = configs.generate_random_sudoku()
    print("Problem Puzzle:")
    print_puzzle(copy.deepcopy(puzzle))
    solver = SudokuCSP(puzzle)
    start_time = time.time()
    solution = solver.lcv_heuristic()
    end_time = time.time()
    times.append(end_time - start_time)
    print(f"Run {i + 1} (Puzzle {i + 1}):")
    if solution:
        solver.print_sudoku(solution)
    else:
        print("No solution found - Puzzle may be invalid")
avg_time = sum(times) / len(times)
print(f"Average time for LCV Heuristic (10 runs): {avg_time:.8f} seconds")

Problem Puzzle:
2 3 1 | 7 5 9 | 6 0 8 
0 7 0 | 0 0 0 | 0 1 0 
6 8 4 | 0 0 1 | 0 0 7 
---------------------
0 6 0 | 2 0 0 | 4 0 5 
9 0 7 | 5 4 0 | 0 0 6 
0 5 0 | 0 0 0 | 0 7 2 
---------------------
7 0 0 | 9 0 0 | 3 0 4 
3 4 6 | 1 7 2 | 8 5 9 
0 0 5 | 0 0 6 | 7 0 0 
Run 1 (Puzzle 1):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - - - - - - -
7 1 2 | 9 8 5 | 3 6 4 
3 4 6 | 1 7 2 | 8 5 9 
8 9 5 | 4 3 6 | 7 2 1 

Problem Puzzle:
2 3 1 | 7 5 9 | 6 0 8 
0 7 0 | 0 0 0 | 0 1 0 
6 8 4 | 0 0 1 | 0 0 7 
---------------------
0 6 0 | 2 0 0 | 4 0 5 
9 0 7 | 5 4 0 | 0 0 6 
0 5 0 | 0 0 0 | 0 7 2 
---------------------
7 0 0 | 9 0 0 | 3 0 4 
3 4 6 | 1 7 2 | 8 5 9 
0 0 5 | 0 0 6 | 7 0 0 
Run 2 (Puzzle 2):
2 3 1 | 7 5 9 | 6 4 8 
5 7 9 | 8 6 4 | 2 1 3 
6 8 4 | 3 2 1 | 5 9 7 
- - - - - - - - - - - -
1 6 3 | 2 9 7 | 4 8 5 
9 2 7 | 5 4 8 | 1 3 6 
4 5 8 | 6 1 3 | 9 7 2 
- - - - - - 